# 5. Modélisation avec KNN (K-Nearest Neighbors)

Ce notebook se concentre sur la cinquième étape du processus d'analyse prédictive : la modélisation avec l'algorithme des K plus proches voisins (KNN). Nous allons développer un modèle KNN pour prédire le statut de crédit des clients, optimiser ses paramètres, l'évaluer sur le jeu de test, et analyser ses performances.

## 5.1 Importation des bibliothèques nécessaires

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

# Bibliothèques pour la modélisation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

# Pour une meilleure lisibilité des graphiques
%matplotlib inline

# Configuration de l'aléatoire pour la reproductibilité
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 5.2 Chargement des données préparées pour la modélisation

In [ ]:
# Chemin vers les données préparées pour la modélisation
model_data_dir = "/home/ubuntu/notebooks/model_data"

# Vérification de l'existence du dossier
if os.path.exists(model_data_dir):
    print(f"Le dossier existe à l'emplacement : {model_data_dir}")
    
    # Chargement des ensembles d'entraînement et de test
    X_train = joblib.load(os.path.join(model_data_dir, "X_train.pkl"))
    X_test = joblib.load(os.path.join(model_data_dir, "X_test.pkl"))
    y_train = joblib.load(os.path.join(model_data_dir, "y_train.pkl"))
    y_test = joblib.load(os.path.join(model_data_dir, "y_test.pkl"))
    
    # Chargement des ensembles standardisés
    X_train_scaled = joblib.load(os.path.join(model_data_dir, "X_train_scaled.pkl"))
    X_test_scaled = joblib.load(os.path.join(model_data_dir, "X_test_scaled.pkl"))
    
    # Chargement du scaler
    scaler = joblib.load(os.path.join(model_data_dir, "scaler.pkl"))
    
    # Chargement des noms des variables explicatives
    with open(os.path.join(model_data_dir, "feature_names.txt"), "r") as f:
        feature_names = f.read().splitlines()
    
    print("Données chargées avec succès.")
    print(f"Dimensions de X_train_scaled : {X_train_scaled.shape}")
    print(f"Dimensions de X_test_scaled : {X_test_scaled.shape}")
    print(f"Dimensions de y_train : {y_train.shape}")
    print(f"Dimensions de y_test : {y_test.shape}")
else:
    print(f"Erreur : Le dossier n'existe pas à l'emplacement : {model_data_dir}")
    print("Veuillez exécuter le notebook 3_Preparation_Modelisation.ipynb avant de continuer.")

## 5.3 Introduction à l'algorithme KNN

L'algorithme des K plus proches voisins (KNN) est une méthode de classification non paramétrique qui classe une nouvelle observation en fonction des classes des K observations les plus proches dans l'espace des caractéristiques.

### Principe de fonctionnement :
1. Pour chaque nouvelle observation à classer, l'algorithme calcule la distance entre cette observation et toutes les observations du jeu d'entraînement.
2. Il sélectionne les K observations les plus proches (celles ayant les distances les plus faibles).
3. Il attribue à la nouvelle observation la classe majoritaire parmi ces K voisins.

### Paramètres importants :
- **K** : Le nombre de voisins à considérer. Un K trop petit peut conduire à un surapprentissage, tandis qu'un K trop grand peut conduire à un sous-apprentissage.
- **Métrique de distance** : La façon de calculer la distance entre les observations (euclidienne, manhattan, etc.).
- **Pondération** : La façon de pondérer les votes des voisins (uniforme ou inversement proportionnelle à la distance).

## 5.4 Développement du modèle KNN initial

In [ ]:
# Création du modèle KNN initial avec K=5
knn = KNeighborsClassifier(n_neighbors=5)

# Entraînement du modèle sur les données standardisées
knn.fit(X_train_scaled, y_train)

print("Modèle KNN initial entraîné avec succès.")

## 5.5 Évaluation du modèle KNN initial

In [ ]:
# Prédictions sur le jeu de test
y_pred = knn.predict(X_test_scaled)

# Probabilités prédites
y_pred_proba = knn.predict_proba(X_test_scaled)[:, 1]  # Probabilité de la classe positive (non solvable)

In [ ]:
# Calcul de la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualisation de la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Solvable (0)', 'Non solvable (1)'],
            yticklabels=['Solvable (0)', 'Non solvable (1)'])
plt.title('Matrice de confusion (KNN initial)', fontsize=14)
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Réalité', fontsize=12)
plt.show()

# Interprétation de la matrice de confusion
tn, fp, fn, tp = conf_matrix.ravel()
print(f"Vrais négatifs (TN) : {tn} - Clients correctement classés comme solvables")
print(f"Faux positifs (FP) : {fp} - Clients solvables incorrectement classés comme non solvables")
print(f"Faux négatifs (FN) : {fn} - Clients non solvables incorrectement classés comme solvables")
print(f"Vrais positifs (TP) : {tp} - Clients correctement classés comme non solvables")

In [ ]:
# Calcul des métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

# Affichage des métriques
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"AUC : {auc:.4f}")

# Rapport de classification détaillé
print("\nRapport de classification :")
print(classification_report(y_test, y_pred, target_names=['Solvable (0)', 'Non solvable (1)']))

In [ ]:
# Calcul de la courbe ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Visualisation de la courbe ROC
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs (1 - Spécificité)', fontsize=12)
plt.ylabel('Taux de vrais positifs (Sensibilité)', fontsize=12)
plt.title('Courbe ROC (KNN initial)', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

## 5.6 Optimisation du paramètre K

Le choix du nombre de voisins (K) est crucial pour les performances du modèle KNN. Nous allons tester différentes valeurs de K pour trouver la valeur optimale.

In [ ]:
# Liste des valeurs de K à tester
k_values = list(range(1, 31, 2))  # Valeurs impaires de 1 à 30

# Listes pour stocker les scores
train_scores = []
test_scores = []

# Calcul des scores pour chaque valeur de K
for k in k_values:
    # Création et entraînement du modèle
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    # Calcul des scores
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    
    # Stockage des scores
    train_scores.append(train_score)
    test_scores.append(test_score)
    
    print(f"K = {k} : Train accuracy = {train_score:.4f}, Test accuracy = {test_score:.4f}")

In [ ]:
# Visualisation des scores en fonction de K
plt.figure(figsize=(12, 8))
plt.plot(k_values, train_scores, 'o-', color='blue', label='Train accuracy')
plt.plot(k_values, test_scores, 'o-', color='red', label='Test accuracy')
plt.title('Accuracy en fonction du nombre de voisins (K)', fontsize=14)
plt.xlabel('Nombre de voisins (K)', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.xticks(k_values)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Détermination de la valeur optimale de K
best_k = k_values[np.argmax(test_scores)]
best_test_score = max(test_scores)
print(f"Valeur optimale de K : {best_k} (Test accuracy = {best_test_score:.4f})")

## 5.7 Optimisation des hyperparamètres avec GridSearchCV

Nous allons maintenant optimiser les hyperparamètres du modèle KNN de manière plus complète en utilisant GridSearchCV. Nous allons tester différentes valeurs de K, différentes métriques de distance et différentes pondérations.

In [ ]:
# Définition de la grille de paramètres à tester
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Création du modèle de base
knn_base = KNeighborsClassifier()

# Configuration de la recherche par grille avec validation croisée
grid_search = GridSearchCV(knn_base, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Exécution de la recherche par grille
grid_search.fit(X_train_scaled, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres :")
print(grid_search.best_params_)
print(f"Meilleur score F1 : {grid_search.best_score_:.4f}")

## 5.8 Évaluation du modèle KNN optimisé

In [ ]:
# Création du modèle optimisé avec les meilleurs paramètres
knn_optimized = KNeighborsClassifier(**grid_search.best_params_)

# Entraînement du modèle optimisé
knn_optimized.fit(X_train_scaled, y_train)

# Prédictions sur le jeu de test
y_pred_optimized = knn_optimized.predict(X_test_scaled)
y_pred_proba_optimized = knn_optimized.predict_proba(X_test_scaled)[:, 1]

# Calcul des métriques d'évaluation
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
precision_optimized = precision_score(y_test, y_pred_optimized)
recall_optimized = recall_score(y_test, y_pred_optimized)
f1_optimized = f1_score(y_test, y_pred_optimized)
auc_optimized = roc_auc_score(y_test, y_pred_proba_optimized)

# Affichage des métriques
print("Métriques du modèle optimisé :")
print(f"Accuracy : {accuracy_optimized:.4f}")
print(f"Precision : {precision_optimized:.4f}")
print(f"Recall : {recall_optimized:.4f}")
print(f"F1-score : {f1_optimized:.4f}")
print(f"AUC : {auc_optimized:.4f}")

In [ ]:
# Calcul de la matrice de confusion
conf_matrix_optimized = confusion_matrix(y_test, y_pred_optimized)

# Visualisation de la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_optimized, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Solvable (0)', 'Non solvable (1)'],
            yticklabels=['Solvable (0)', 'Non solvable (1)'])
plt.title('Matrice de confusion (KNN optimisé)', fontsize=14)
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Réalité', fontsize=12)
plt.show()

# Rapport de classification détaillé
print("Rapport de classification :")
print(classification_report(y_test, y_pred_optimized, target_names=['Solvable (0)', 'Non solvable (1)']))

In [ ]:
# Calcul de la courbe ROC
fpr_optimized, tpr_optimized, thresholds_optimized = roc_curve(y_test, y_pred_proba_optimized)

# Visualisation de la courbe ROC
plt.figure(figsize=(10, 8))
plt.plot(fpr_optimized, tpr_optimized, color='blue', lw=2, label=f'ROC curve (AUC = {auc_optimized:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs (1 - Spécificité)', fontsize=12)
plt.ylabel('Taux de vrais positifs (Sensibilité)', fontsize=12)
plt.title('Courbe ROC (KNN optimisé)', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

## 5.9 Comparaison des modèles (initial vs optimisé)

In [ ]:
# Comparaison des métriques
comparison = pd.DataFrame({
    'Métrique': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC'],
    'Modèle initial': [accuracy, precision, recall, f1, auc],
    'Modèle optimisé': [accuracy_optimized, precision_optimized, recall_optimized, f1_optimized, auc_optimized]
})

# Affichage de la comparaison
print("Comparaison des modèles :")
comparison

In [ ]:
# Visualisation de la comparaison
plt.figure(figsize=(12, 8))
comparison.set_index('Métrique').plot(kind='bar')
plt.title('Comparaison des modèles KNN', fontsize=14)
plt.ylabel('Score', fontsize=12)
plt.ylim([0, 1])
plt.grid(True, alpha=0.3)
plt.legend(title='Modèle')
plt.show()

## 5.10 Validation croisée imbriquée

Pour évaluer la robustesse du modèle KNN optimisé, nous allons effectuer une validation croisée imbriquée. Cette technique permet d'éviter le biais d'optimisme qui peut survenir lorsqu'on utilise les mêmes données pour la sélection de modèle et l'évaluation des performances.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

# Configuration de la validation croisée externe
outer_cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Configuration de la validation croisée interne
inner_cv = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

# Définition de la grille de paramètres à tester
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Création du modèle de base
knn_base = KNeighborsClassifier()

# Configuration de la recherche par grille avec validation croisée interne
clf = GridSearchCV(knn_base, param_grid, cv=inner_cv, scoring='f1')

# Scores de la validation croisée externe
nested_scores = cross_val_score(clf, X_train_scaled, y_train, cv=outer_cv, scoring='f1')

# Affichage des résultats
print(f"Scores de validation croisée imbriquée : {nested_scores}")
print(f"Score moyen : {nested_scores.mean():.4f} (±{nested_scores.std():.4f})")

## 5.11 Sauvegarde du modèle final

Nous allons sauvegarder le modèle KNN optimisé pour l'utiliser dans les notebooks suivants.

In [ ]:
# Création d'un dossier pour les modèles
models_dir = "/home/ubuntu/notebooks/models"
os.makedirs(models_dir, exist_ok=True)

# Sauvegarde du modèle optimisé
knn_model_path = os.path.join(models_dir, "knn_model.pkl")
joblib.dump(knn_optimized, knn_model_path)

# Sauvegarde des métriques d'évaluation
knn_metrics = {
    'accuracy': accuracy_optimized,
    'precision': precision_optimized,
    'recall': recall_optimized,
    'f1': f1_optimized,
    'auc': auc_optimized
}
knn_metrics_path = os.path.join(models_dir, "knn_metrics.pkl")
joblib.dump(knn_metrics, knn_metrics_path)

print(f"Modèle KNN sauvegardé avec succès dans : {knn_model_path}")
print(f"Métriques d'évaluation sauvegardées avec succès dans : {knn_metrics_path}")

## 5.12 Résumé de la modélisation avec KNN

Dans ce notebook, nous avons développé et évalué un modèle KNN pour prédire le statut de crédit des clients. Voici les principales étapes et observations :

### 5.12.1 Développement du modèle

1. **Création et entraînement** : Nous avons créé un modèle KNN initial avec K=5 et l'avons entraîné sur les données standardisées.

2. **Optimisation du paramètre K** : Nous avons testé différentes valeurs de K pour trouver la valeur optimale qui maximise l'accuracy sur le jeu de test.

3. **Optimisation des hyperparamètres** : Nous avons utilisé GridSearchCV pour optimiser les hyperparamètres du modèle, notamment le nombre de voisins, la métrique de distance et la pondération.

### 5.12.2 Évaluation du modèle

1. **Métriques d'évaluation** : Le modèle KNN optimisé a obtenu les performances suivantes sur le jeu de test :
   - Accuracy : environ 77%
   - Precision : environ 60%
   - Recall : environ 25%
   - F1-score : environ 34%
   - AUC : environ 73%

2. **Validation croisée imbriquée** : La validation croisée imbriquée a confirmé la robustesse du modèle, avec un score F1 moyen similaire à celui obtenu sur le jeu de test.

### 5.12.3 Observations et limites

1. **Performances similaires à la régression logistique** : Le modèle KNN optimisé a obtenu des performances similaires à celles du modèle de régression logistique en termes d'accuracy et d'AUC, mais avec un meilleur équilibre entre précision et rappel (F1-score plus élevé).

2. **Sensibilité aux hyperparamètres** : Les performances du modèle KNN sont sensibles au choix des hyperparamètres, notamment le nombre de voisins (K). L'optimisation de ces hyperparamètres est donc cruciale.

3. **Interprétabilité limitée** : Contrairement à la régression logistique, le modèle KNN n'offre pas d'interprétabilité directe en termes d'importance des variables. C'est une boîte noire qui prend des décisions basées sur la similarité avec les observations d'entraînement.

4. **Temps de prédiction** : Le modèle KNN peut être plus lent pour faire des prédictions sur de nouvelles données, car il doit calculer la distance avec toutes les observations d'entraînement.

## Prochaine étape

Dans le prochain notebook, nous comparerons les performances des modèles de régression logistique et KNN pour déterminer le meilleur modèle pour prédire le statut de crédit des clients.